In [ ]:
import numpy as np
import sklearn 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from scipy.stats import norm
from scipy.optimize import minimize 
import random
import time
import timeit
import matplotlib.pyplot as plt

import pyswarms as ps
from pyswarms.discrete import binary as bi

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
#import filetxt and assign sample x and sample y

batch_num = 3
num_bits = 50
initial_path = f"D:/Thickness_optimization/Thickness_optimization_binary_thickness/Initial_data/bat{batch_num}/"
print(initial_path)
filename='training_data_thickness_binary_sample{n}_{bits}'.format(n=batch_num,bits = num_bits+6)
f_1=open(initial_path+filename+'.txt')
data_array=np.loadtxt(f_1)
sample_x=np.float32(data_array[:,1:data_array.shape[1]])
sample_y=np.float32(data_array[:,0])

print(sample_x)

In [ ]:
#Defining Optimizer

def GPRoptimizer(obj_func, initial_theta, bounds):
   
    def get_fun(theta): 
   #reshape X_star to fit the later input
        fun_v=obj_func(theta, eval_gradient=True)
   #print(str(fun_v))
        return fun_v
   
    funval = minimize(get_fun, initial_theta, method='L-BFGS-B', jac=True, bounds=bounds,options={'maxiter':30000})
    theta_opt=funval.x
   
    def get_fun_eval(theta):
   #reshape X_star to fit the later input
       fun_eval=obj_func(theta, eval_gradient=False)
       return fun_eval
    func_min=get_fun_eval(theta_opt)
    return theta_opt, func_min

kernel = 1 * RBF(length_scale=1, length_scale_bounds=(1e-2, 1e2))
#-------------------------------------------------------------------------------

gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=1,optimizer=GPRoptimizer)
gaussian_process.fit(sample_x, sample_y)
gaussian_process.kernel_

In [ ]:
#Defining Acquasition Function <one input>

def acq_fun_EI_binary(x_star):
    y_sample_data = sample_y
    y_prediction_mean, y_prediciton_var = gaussian_process.predict(x_star, return_std=True) #x_star[binary]
    
    def EI(mean, var, ff):
        Z = (ff.min() - mean) / np.sqrt(var)
        ei = (ff.min() - mean) * norm.cdf(Z) + np.sqrt(var) * norm.pdf(Z)
        return ei
    ei = EI(y_prediction_mean, y_prediciton_var, y_sample_data)
    return -ei 
#mean_prediction, std_prediction = gaussian_process.predict(current_x_junk, return_std=True)

In [ ]:
#PSO Search Digital Space

qv_vector_size=int(sample_x.shape[1])  #가로축 길이 : 16
x_min_junk_container=np.zeros([1,qv_vector_size]) #[1,16] matrix (x+1 저장)
fom_min_junk_container = np.zeros(1) #(y+1 저장)

In [ ]:
#PSO option 

#set-up hyperparameters
BPSO_part = 2**6
BPSO_iter = 2**13
options = {'c1':0.5,'c2':0.3,'w':0.9,'k':BPSO_part,'p':1}


In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [ ]:
#matlab 함수 이용해서 다음 FOM값 찾기


history_FOM = [] #history 그래프를 그리기 위해, 각 iter당 min값 저장
history_x = []
iteration_BO = 2000

for i in range(1,iteration_BO+1):
    
    
    print("\n iteration: ",i)
    start_time = time.time()
    
    #Call instace of PSO (single.GlobalBest used)
    start_time_PSO = time.time()
    optimizer = bi.BinaryPSO(n_particles=BPSO_part, dimensions=qv_vector_size, options=options)
    #Perform optimization
    cost, pos_binary = optimizer.optimize(acq_fun_EI_binary,iters=BPSO_iter,n_processes=None, verbose = False)
    end_time_PSO = time.time()

    execution_time_PSO = round((end_time_PSO - start_time_PSO),3)
    print(f"Execution time PSO: {execution_time_PSO} seconds")

    x_initial_guess = pos_binary
    print("x+1 : ",x_initial_guess)
    
    for k in range(sample_x.shape[0]):
        if(np.array_equal(x_initial_guess[:],sample_x[k,:])):
            print("random!: ",i)
            for j in range(qv_vector_size):
                qv_random = random.random()
                if (qv_random>1/2):
                    x_initial_guess[j]=1
                else:
                    x_initial_guess[j]=0

    sample_x=np.vstack((sample_x,x_initial_guess)) #sample_x에 (x+1)추가
    x_list = x_initial_guess.tolist() #array를 list로 변환, matlab engine 구동시 필요
    
    for ind in range(qv_vector_size):
        x_list[ind]=int(x_list[ind])

    x_mat = matlab.int16(x_list)

    #-----------------------here TMM-------------------------------------------
    start_time_TMM = time.time()
    fom_min_junk_container[0]=eng.adaptive_scheme_FOM(x_mat)  #matlab으로 계산한 x+1의 fom값
    end_time_TMM = time.time()
    execution_time_TMM = round((end_time_TMM - start_time_TMM),3)

    #------------------------------------------------------------------
    
    print("for x+1, fom: ", fom_min_junk_container[0])
    sample_y=np.append(sample_y,np.array([fom_min_junk_container[0]]))

    #----------------------------here GP------------------------------
    start_time_GP = time.time()
    gaussian_process.fit(sample_x, sample_y)
    end_time_GP = time.time()
    execution_time_GP = round((end_time_GP - start_time_GP),3)

    print(f"Execution time Gaussian: {execution_time_GP} seconds")
    #------------------------------------------------------------------
    
    history_FOM=np.append(history_FOM,sample_y.min())
    min_fom_x = sample_x[sample_y.argmin(),:]
    history_x.append(min_fom_x)
    
    print("minimum fom address: ",min_fom_x)
    print("minimum fom: ",sample_y.min())
    
    
     #--------------------save_result_txt------------------------------
    result2_str = ' '.join(map(str, x_list))
    result1_str = f"{fom_min_junk_container[0]:1.15f}"
    save_fom = '{size}_layers_{iters}_iters_GPR_PSO_thickness_binary_{n}_{part}_{iterPSO}'.format(size = num_bits, iters = iteration_BO,n=batch_num,part = int(np.log2(BPSO_part)),iterPSO = int(np.log2(BPSO_iter)))
    file_path = f'D:/Thickness_optimization/Thickness_optimization_binary_thickness/BO_PSO_binary_opt_result/bat{batch_num}/'
    
    if (i==1):
        sample_y_reshaped = sample_y.reshape(-1, 1)
        merged_array = np.hstack((sample_y_reshaped, sample_x))
        N = sample_x.shape[1]
        format_txt=' '.join(['%1.15f'] + ['%d']*N)
        np.savetxt(file_path+save_fom+'.txt', merged_array, fmt = format_txt, delimiter=' ')
        
    else:
        with open(file_path+save_fom+'.txt','a') as writing_txt:
            writing_txt.write(f"{result1_str} {result2_str}\n")
        
    end_time = time.time()
    time_record = int(end_time - start_time)
    print("total time: ",time_record,'sec')
    save_time = '{size}_layers_{iters}_iters_time_record_GPR_PSO_thickness_binary_{n}_{part}_{iterPSO}'.format(size = num_bits, iters = iteration_BO,n=batch_num,part = int(np.log2(BPSO_part)),iterPSO = int(np.log2(BPSO_iter)))
    
    with open(file_path+save_time+'.txt','a') as time_txt:
        time_txt.write(f"{i} {execution_time_GP} {execution_time_PSO} {execution_time_TMM} {time_record}\n")
    